In [1]:
import torch

def default_device():
    if torch.cuda.is_available():
        return torch.device('cuda')   
    if torch.backends.mps.is_available():
        return torch.device('mps')
    return torch.device('cpu')

device = default_device()

device(type='mps')

### PyTorch 搭建与使用神经网络

数据 -> 网络结构 -> 损失函数 -> 优化器 -> 训练 -> 测试 -> 保存模型 

-> 加载模型 -> 预测

In [2]:
from torchvision import datasets, transforms
import  torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

#### 加载数据

In [ ]:
train_data = datasets.MNIST('data', train=True, download=True, transform=transforms.ToTensor())
test_data = datasets.MNIST('data', train=False, download=True, transform=transforms.ToTensor())

In [4]:
batch_size = 100
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

#### 定义网络

In [5]:
class MLP(nn.Module):
    def __init__(self, input_dim=28*28, hidden_dim=512, output_dim=10):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, output_dim)


        # self.model = nn.Sequential(
        #     nn.Linear(28*28, 512),
        #     nn.ReLU(),
        #     nn.Linear(512, 256),
        #     nn.ReLU(),
        #     nn.Linear(256, 10)
        # )
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.relu(out)
        out = self.fc3(out)
        return out

input_dim = 28*28
hidden_dim = 512
output_dim = 10

model = MLP().to(device)

#### 损失函数

In [6]:
criterion = nn.CrossEntropyLoss()

#### 优化器

In [7]:
learning_rate = 0.001
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

#### 训练

In [ ]:
num_epochs = 10

for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.view(-1, 28*28).to(device)
        labels = labels.to(device)
        outputs = model(images)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print(f'Epoch: {epoch+1}, Step: {i+1}, Loss: {loss.item()}')

#### 测试

In [ ]:
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.view(-1, 28*28).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    print(f'Accuracy: {100*correct/total}')

#### 保存

In [10]:
torch.save(model, 'model/mnist_mlp.pth')